### Testing Notebook while Developing

In [1]:
from video_cv_project.models.encoders import create_resnet_encoder
from video_cv_project.models.crw import CRW

encoder = create_resnet_encoder(del_layers=['layer4'])

In [2]:
model = CRW(encoder, edge_dropout=0.0, feat_dropout=0.0, temperature=0.07, head_depth=1)

In [3]:
from torchinfo import summary

# Dataloader output tensor: NCTHW
summary(encoder, input_size=(1, 3, 8, 640, 640))

Layer (type:depth-idx)                        Output Shape              Param #
VideoWrapper                                  [1, 256, 8, 80, 80]       --
├─ResNet: 1-1                                 [8, 256, 80, 80]          --
│    └─Conv2d: 2-1                            [8, 64, 320, 320]         9,408
│    └─BatchNorm2d: 2-2                       [8, 64, 320, 320]         128
│    └─ReLU: 2-3                              [8, 64, 320, 320]         --
│    └─MaxPool2d: 2-4                         [8, 64, 160, 160]         --
│    └─Sequential: 2-5                        [8, 64, 160, 160]         --
│    │    └─BasicBlock: 3-1                   [8, 64, 160, 160]         73,984
│    │    └─BasicBlock: 3-2                   [8, 64, 160, 160]         73,984
│    └─Sequential: 2-6                        [8, 128, 80, 80]          --
│    │    └─BasicBlock: 3-3                   [8, 128, 80, 80]          230,144
│    │    └─BasicBlock: 3-4                   [8, 128, 80, 80]          295,42

In [4]:
summary(model, input_size=(1, 8, 192, 64, 64))

Layer (type:depth-idx)                             Output Shape              Param #
CRW                                                [1, 128, 8, 64]           --
├─VideoWrapper: 1-1                                [64, 256, 8, 8, 8]        --
│    └─ResNet: 2-1                                 [512, 256, 8, 8]          --
│    │    └─Conv2d: 3-1                            [512, 64, 32, 32]         9,408
│    │    └─BatchNorm2d: 3-2                       [512, 64, 32, 32]         128
│    │    └─ReLU: 3-3                              [512, 64, 32, 32]         --
│    │    └─MaxPool2d: 3-4                         [512, 64, 16, 16]         --
│    │    └─Sequential: 3-5                        [512, 64, 16, 16]         147,968
│    │    └─Sequential: 3-6                        [512, 128, 8, 8]          525,568
│    │    └─Sequential: 3-7                        [512, 256, 8, 8]          2,099,712
│    │    └─Identity: 3-8                          [512, 256, 8, 8]          --
│    │    └─Id

In [11]:
import torch

feats, loss, debug = model(torch.rand(1, 8, 3, 640, 640))

In [1]:
from video_cv_project.data.transform import create_train_pipeline
import torchvision.transforms.functional as F
from PIL import Image

transform = create_train_pipeline()
im = Image.open('./test.png')
im = F.to_tensor(im)[:3]
vid = im.repeat(8, 1, 1, 1)
print(transform)
print(vid.shape)
print(transform(vid).shape)

MapTransform(
    ToPILImage()
    Resize(size=(256, 256), interpolation=lanczos, max_size=None, antialias=None)
    ToTensor()
    PatchSplitTransform(size=(64, 64), stride=(32, 32))
    MapTransform(
        ToPILImage()
        RandomResizedCrop(size=(64, 64), scale=(0.8, 0.95), ratio=(1.0, 1.0), interpolation=lanczos), antialias=None)
        ToTensor()
        Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
    )
)
torch.Size([8, 3, 480, 480])
torch.Size([8, 49, 3, 64, 64])
